### Project Overview:

The goal of this project is to perform Extract, Transform, and Load (ETL) operations on a sales dataset. The dataset likely contains information about sales transactions, including details such as region, country, item type, order date etc.

ETL processes involve extracting data from various sources, transforming it into a suitable format, and loading it into a destination such as a database for further analysis or reporting. In this project, we'll use Python along with libraries like pandas for data manipulation, scipy for statistical analysis, and sqlalchemy for connecting to a PostgreSQL database.

The overall objective is to clean, preprocess, and structure the sales data, ensuring it is in a usable format for analysis and reporting purposes. This involves tasks such as handling missing values, removing outliers, converting data types, and loading the cleaned data into a PostgreSQL database for storage and further analysis.

Import the necessary libraries for data manipulation and statistical analysis, and database connection, then read the dataset form the CSV file:

1. pandas: To read and manipulate CSV files
2. scipy.stats: For outlier detection

In [1]:
import pandas as pd
from scipy import stats

C:\Users\biny\AppData\Local\Temp\ipykernel_12052\2233648990.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv("sales-records.csv")
df.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Sub-Saharan Africa,South Africa,Fruits,Offline,M,7/27/2012,443368995,7/28/2012,1593,9.33,6.92,14862.69,11023.56,3839.13
1,Middle East and North Africa,Morocco,Clothes,Online,M,9/14/2013,667593514,10/19/2013,4611,109.28,35.84,503890.08,165258.24,338631.84
2,Australia and Oceania,Papua New Guinea,Meat,Offline,M,5/15/2015,940995585,6/4/2015,360,421.89,364.69,151880.40,131288.40,20592.00
3,Sub-Saharan Africa,Djibouti,Clothes,Offline,H,5/17/2017,880811536,7/2/2017,562,109.28,35.84,61415.36,20142.08,41273.28
4,Europe,Slovakia,Beverages,Offline,L,10/26/2016,174590194,12/4/2016,3973,47.45,31.79,188518.85,126301.67,62217.18


In [8]:
print("Item types available in the dataset: ", df["Item Type"].unique(), "\n")
print("Regions in the dataset: ", df["Region"].unique(), "\n")
print("Sales channels in the dataset: ", df["Sales Channel"].unique(), "\n")
print("Order priority types in the dataset: ", df["Order Priority"].unique(), "\n")

Item types available in the dataset:  ['Fruits' 'Clothes' 'Meat' 'Beverages' 'Office Supplies' 'Cosmetics'
 'Snacks' 'Personal Care' 'Household' 'Vegetables' 'Baby Food' 'Cereal'] 

Regions in the dataset:  ['Sub-Saharan Africa' 'Middle East and North Africa'
 'Australia and Oceania' 'Europe' 'Asia'
 'Central America and the Caribbean' 'North America'] 

Sales channels in the dataset:  ['Offline' 'Online'] 

Order priority types in the dataset:  ['M' 'H' 'L' 'C'] 



In [9]:
df.shape

(1000000, 14)

As we can see there are exactly 1 Million rows in the data set. Now lets check and remove missiong/null or duplicated values

In [13]:
missing_data = df.isnull().sum()
missing_data

Region            0
Country           0
Item Type         0
Sales Channel     0
Order Priority    0
Order Date        0
Order ID          0
Ship Date         0
Units Sold        0
Unit Price        0
Unit Cost         0
Total Revenue     0
Total Cost        0
Total Profit      0
dtype: int64

There are no missing/null values

Now lets look for duplicate values.

In [17]:
duplicate_rows = df.duplicated()

# Remove duplicate rows
cleaned_data = df.drop_duplicates()

df.shape


(941278, 14)

Now lets try to normalize the numerical data by removing outlier data.

In [18]:
# Loop through numerical columns
for column in df.select_dtypes(include='number').columns:
    z_scores = stats.zscore(df[column])

    # Set threshold for outlier detection
    threshold = 3
    outlier_indices = (z_scores > threshold) | (z_scores < -threshold)

    # Remove outliers
    df = df[~outlier_indices]


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 895970 entries, 0 to 999999
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Region          895970 non-null  object 
 1   Country         895970 non-null  object 
 2   Item Type       895970 non-null  object 
 3   Sales Channel   895970 non-null  object 
 4   Order Priority  895970 non-null  object 
 5   Order Date      895970 non-null  object 
 6   Order ID        895970 non-null  int64  
 7   Ship Date       895970 non-null  object 
 8   Units Sold      895970 non-null  int64  
 9   Unit Price      895970 non-null  float64
 10  Unit Cost       895970 non-null  float64
 11  Total Revenue   895970 non-null  float64
 12  Total Cost      895970 non-null  float64
 13  Total Profit    895970 non-null  float64
dtypes: float64(5), int64(2), object(7)
memory usage: 102.5+ MB


In [20]:
df.describe()

,Order ID,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
count,8.959700e+05,895970.000000,895970.000000,895970.000000,8.959700e+05,8.959700e+05,8.959700e+05
mean,5.494291e+08,4620.037658,230.379009,159.407136,9.376746e+05,6.362917e+05,3.013829e+05
std,2.599282e+08,2787.854051,197.284868,159.344168,9.019418e+05,7.014165e+05,2.613479e+05
min,1.000012e+08,1.000000,9.330000,6.920000,9.330000e+00,6.920000e+00,2.410000e+00
25%,3.239670e+08,2240.000000,81.730000,35.840000,2.344834e+05,1.365154e+05,8.029500e+04
50%,5.488118e+08,4480.000000,154.060000,97.440000,6.627654e+05,3.696017e+05,2.304062e+05
75%,7.746390e+08,6864.000000,421.890000,263.330000,1.374938e+06,8.728208e+05,4.751604e+05
max,9.999999e+08,10000.000000,668.270000,524.960000,4.182034e+06,3.145087e+06,1.149455e+06


In order to store the dataset into postgres database we need to change the data format in the `Order Date` and `Ship Date` columns into postgres compatible format. (From `7/27/2012` to `2012-7-27`).

In [21]:
# Convert date column to datetime format
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

# Convert datetime format to PostgreSQL-compatible date format
df['Order Date'] = df['Order Date'].dt.strftime('%Y-%m-%d')
df['Ship Date'] = df['Ship Date'].dt.strftime('%Y-%m-%d')

# Print the first few rows to verify the conversion
print(df.head())

                         Region           Country  Item Type Sales Channel  \
0            Sub-Saharan Africa      South Africa     Fruits       Offline   
1  Middle East and North Africa           Morocco    Clothes        Online   
2         Australia and Oceania  Papua New Guinea       Meat       Offline   
3            Sub-Saharan Africa          Djibouti    Clothes       Offline   
4                        Europe          Slovakia  Beverages       Offline   

  Order Priority  Order Date   Order ID   Ship Date  Units Sold  Unit Price  \
0              M  2012-07-27  443368995  2012-07-28        1593        9.33   
1              M  2013-09-14  667593514  2013-10-19        4611      109.28   
2              M  2015-05-15  940995585  2015-06-04         360      421.89   
3              H  2017-05-17  880811536  2017-07-02         562      109.28   
4              L  2016-10-26  174590194  2016-12-04        3973       47.45   

   Unit Cost  Total Revenue  Total Cost  Total Profit  


Now the data set is ready to be loaded into the database.